# Spark Assignment
* Xuyan Xiao, xx2226
* Junhui Liao, jl4574

### Data Preparation

In [1]:
# user and artist
rawUserArtistData = sc.textFile("user_artist_data.txt")

# use a sample to train the model
# weights = [.1, .9]
# seed = 42
# rawUserArtistData, someOtherJunk = rawUserArtistData.randomSplit(weights, seed)
# rawUserArtistData.cache()

# print rawUserArtistData.map(lambda l: float(l.split(" ")[0])).stats()
# print rawUserArtistData.map(lambda l: float(l.split(" ")[1])).stats()

In [2]:
# artist data
rawArtistData = sc.textFile("artist_data.txt")

def lineSplit(artist):
    line = artist.split("\t")
    if len(line) < 2:
        return []
    else:
        try:
            return [(int(line[0]),line[1].strip())]
        except:
            return []

artistByID = rawArtistData.flatMap(lambda l: lineSplit(l))

In [3]:
# artist misspelled
rawArtistAlias = sc.textFile("artist_alias.txt")

def lineSplitAlias(artist):
    line = artist.split("\t",1)
    if line[0] == "":
        return []
    else:
        return [(int(line[0]),int(line[1]))]

artistAlias = rawArtistAlias.flatMap(lambda l: lineSplitAlias(l)).collectAsMap()

In [14]:
print artistByID.lookup(6803336)[0]
print artistByID.lookup(1000010)[0]

Aerosmith (unplugged)
Aerosmith


### build the first recommendation model

In [4]:
from pyspark.mllib.recommendation import *
bArtistAlias = sc.broadcast(artistAlias)

def trainDataConstuct(line):
    userID, artistID, count = map(int,line.split(" "))
    finalArtistID = bArtistAlias.value.get(artistID,artistID)
    return Rating(userID, finalArtistID, count)

trainData = rawUserArtistData.map(lambda l: trainDataConstuct(l))
trainData.cache()

PythonRDD[7] at RDD at PythonRDD.scala:43

In [5]:
model = ALS.trainImplicit(trainData, 10, 5, 0.01)

In [11]:
# look at the first line of the model
# ft = model.userFeatures().collect()
# (ft[0][0],)+tuple([it for it in ft[0][1]])

rawArtistsForUser = trainData.filter(lambda u: u.user == 2093760)

# take a look at all the artists the given user has listened to
# t = rawArtistsForUser.collect()
# print t

# find the unique artisits
existingProducts = set(rawArtistsForUser.map(lambda m: int(m.product)).collect())

# print existingProducts
arts = artistByID.filter(lambda l: l[0] in existingProducts).values().collect()
for art in arts:
    print art

David Gray
Blackalicious
Jurassic 5
The Saw Doctors
Xzibit


In [12]:
# recommendations = model.recommendProducts(1000002, 10) not included in pysprak 1.3.1
recommendations = model.call("recommendProducts", 2093760, 10)
recommendedProductIDs = set([item.product for item in recommendations])

In [13]:
# match with the artists' names and print out
res = artistByID.filter(lambda l: l[0] in recommendedProductIDs).values().collect()
for artist in res:
    print artist

50 Cent
Snoop Dogg
Nas
Jay-Z
Kanye West
Dr. Dre
2Pac
Eminem
Outkast
The Roots
